<a href="https://colab.research.google.com/github/greypen2006/Final_Project/blob/main/N3.%20Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Merging Indicator Data**

In [1]:
#This tells python to draw the graphs "inline" - in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
import statistics
import seaborn as sns
from IPython.display import display, Math, Latex, display_latex
import plotly.express as px
import pylab
import pandas as pd
import numpy as np
# make the plots (graphs) a little wider by default
pylab.rcParams['figure.figsize'] = (10., 8.)
sns.set(font_scale=1.5)
sns.set_style("white")

  **Outline**

1. Clean and identify indeces to merge data with
2. Upload all csv for indicators to use in the OLS
3. Create and download a new merged df that can be used for future analysis.

In [2]:
# As seen in cleaning, population.csv will be downloaded with google.colab files, other with mkdir

In [3]:
from google.colab import files
uploaded=files.upload()


Saving Population.csv to Population.csv


In [4]:
#opening the files to find similar columns to use as indeces
sex_df=pd.read_csv('Population.csv', skiprows=35, nrows=13)
sex_df.head()

,local authority: county / unitary (as of April 2023),mnemonic,2017,2018,2019,2020,2021,2022,2023,2024
0,Brent,E09000005,172035.0,174420.0,175791.0,174877.0,172954.0,174856.0,178703.0,180473.0
1,Camden,E09000007,112745.0,112981.0,112990.0,112281.0,111280.0,115471.0,115778.0,115907.0
2,Greenwich,E09000011,144020.0,145791.0,147386.0,147838.0,148738.0,150103.0,152034.0,153409.0
3,Hackney,E09000012,136757.0,137823.0,137858.0,137274.0,135851.0,137030.0,138971.0,139603.0
4,Hammersmith and Fulham,E09000013,97539.0,98619.0,98936.0,98125.0,97403.0,99119.0,100386.0,101423.0


In [5]:
!mkdir -p data
!curl -L "https://drive.google.com/uc?export=download&id=10lDwtL9VawYDqiaM4hhP9ASP4PHSDf5H" -o ./data/rent_prices.csv
rent_df=pd.read_csv('./data/rent_prices.csv')
rent_df.head()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   765  100   765    0     0    373      0  0:00:02  0:00:02 --:--:--  1416


,Borough,Mean_Monthly_Rent,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021,2021-2022,2022-2023,2023-2024,2024-2025
0,Brent,1677,1558,1561,1575,1498,NaN,1482,1578,2156,2006
1,Camden,2238,2053,1939,2416,2218,NaN,2076,2383,2202,2615
2,Greenwich,1539,1370,1435,1385,1479,NaN,1478,1557,1740,1866
3,Hackney,1958,1757,1825,1841,1831,NaN,1746,1997,2191,2478
4,Hammersmith and Fulham,2138,1975,1737,2055,2115,NaN,2019,2270,2418,2516


In [6]:
!mkdir -p data
!curl -L "https://drive.google.com/uc?export=download&id=1iStDMn2Vh_FvlvLhDfx-d4nXNw14_XPx" -o ./data/weekly_income_cleaned.csv
income_df=pd.read_csv('./data/weekly_income_cleaned.csv')
income_df.head()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   704  100   704    0     0    315      0  0:00:02  0:00:02 --:--:--   367


,Year,Brent,Camden,Greenwich,Hackney,Hammersmith and Fulham,Islington,Kensington and Chelsea,Lambeth,Southwark,Tower Hamlets,Westminster
0,2017,471.0,614.3,553.0,558.1,651.1,646.1,784.4,594.6,587.1,592.0,730.8
1,2018,479.9,634.4,566.0,555.4,682.0,687.7,670.6,617.1,589.4,627.6,757.1
2,2019,524.0,662.0,579.5,574.9,728.0,699.1,763.7,634.8,628.0,688.9,703.9
3,2020,531.8,623.0,573.1,574.9,713.1,690.1,802.0,670.8,620.9,665.0,701.1
4,2021,555.7,695.0,618.3,576.8,727.0,670.7,763.6,681.2,637.8,685.2,771.1


In [7]:
!mkdir data
!curl -L "https://drive.google.com/uc?export=download&id=1mquBNjbupyPX1a1iHfGBCbycC91mKQih" -o ./data/PubCount.csv
pub_df=pd.read_csv('./data/PubCount.csv')
pub_df.head()

mkdir: cannot create directory ‘data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   484  100   484    0     0    291      0  0:00:01  0:00:01 --:--:--  1613


,BoroughName,2017,2018,2019,2020,2021,2022,2023
0,Brent,70,65,65,55,60,70,70
1,Camden,245,240,235,245,240,240,235
2,Greenwich,75,65,75,65,65,75,70
3,Hackney,235,225,200,190,200,205,205
4,Hammersmith and Fulham,95,90,90,100,100,100,95


In [8]:
!mkdir -p data
!curl -L "https://drive.google.com/uc?export=download&id=1Dy-eciO2EXLkgbmP1FRoQrYt9p_qkMj6" -o ./data/crime_cleaned_final.csv
crime_df=pd.read_csv('./data/crime_cleaned_final.csv')
crime_df.head()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3364k  100 3364k    0     0  1524k      0  0:00:02  0:00:02 --:--:-- 4744k


,MajorText,MinorText,BoroughName,YearMonth,CrimeCount,Year,mnemonic,Population,CrimeRate
0,ARSON AND CRIMINAL DAMAGE,ARSON,Brent,201701,3,2017,E09000005,342439,0.000876
1,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,Brent,201701,155,2017,E09000005,342439,0.045264
2,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Brent,201701,56,2017,E09000005,342439,0.016353
3,BURGLARY,DOMESTIC BURGLARY,Brent,201701,195,2017,E09000005,342439,0.056944
4,BURGLARY,RES BURGLARY OF A HOME,Brent,201701,0,2017,E09000005,342439,0.000000


In [9]:
# Filter for just sexual crimes
sexual_crime_df = crime_df[crime_df['MajorText'] == 'SEXUAL OFFENCES']
sexual_crime_df.head()

,MajorText,MinorText,BoroughName,YearMonth,CrimeCount,Year,mnemonic,Population,CrimeRate
17,SEXUAL OFFENCES,OTHER SEXUAL OFFENCES,Brent,201701,26,2017,E09000005,342439,0.007593
18,SEXUAL OFFENCES,RAPE,Brent,201701,15,2017,E09000005,342439,0.004380
49,SEXUAL OFFENCES,OTHER SEXUAL OFFENCES,Camden,201701,27,2017,E09000007,218833,0.012338
50,SEXUAL OFFENCES,RAPE,Camden,201701,13,2017,E09000007,218833,0.005941
81,SEXUAL OFFENCES,OTHER SEXUAL OFFENCES,Greenwich,201701,22,2017,E09000011,282376,0.007791


In [10]:
!curl -L "https://drive.google.com/uc?export=download&id=1lgnCQt8DJxsJj6kIGMmQ1AKaGufzKoeM" -o ./data/unemployment_cleaned.csv
job_df=pd.read_csv('./data/unemployment_cleaned.csv')
job_df.head()
#

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1320  100  1320    0     0    721      0  0:00:01  0:00:01 --:--:--  2603


,local authority: district / unitary (as of April 2023),mnemonic,Jul 2016-Jun 2017,Conf,Jul 2017-Jun 2018,Conf.1,Jul 2018-Jun 2019,Conf.2,Jul 2019-Jun 2020,Conf.3,Jul 2020-Jun 2021,Conf.4,Jul 2021-Jun 2022,Conf.5,Jul 2022-Jun 2023,Conf.6,Jul 2023-Jun 2024,Conf.7,Jul 2024-Jun 2025,Conf.8
0,Brent,E09000005,6.6,1.5,5.9,1.4,4.1,1.4,5.9,1.9,7.6,2.1,5.4,1.8,7.3,3.1,5.5,2.3,8.2,3.0
1,Camden,E09000007,5.3,1.3,5.0,1.4,4.8,1.6,4.0,1.4,5.3,1.7,3.8,1.4,4.0,1.7,6.3,2.2,5.0,2.4
2,Greenwich,E09000011,6.1,1.4,5.5,1.3,4.3,1.3,4.9,1.4,6.6,1.5,5.9,1.5,4.7,1.5,4.1,1.4,6.4,2.3
3,Hackney,E09000012,5.7,1.4,5.7,1.4,5.0,1.5,5.4,1.6,6.1,1.8,4.3,1.5,4.3,1.7,4.4,1.7,5.3,1.8
4,Hammersmith and Fulham,E09000013,5.8,1.4,5.6,1.5,5.8,1.8,4.5,1.5,5.8,1.7,4.7,1.7,3.9,1.7,5.7,2.1,4.3,1.9


# Merging Data
`sex_df`, `rent_df`, `income_df`, `pub_df`, `sexual_crime_df`, and `job_df` need to be merged into a single master dataframe: `merged_df` to carry out OLS.

*   **`sex_df`**:
    *   Rename 'Borough' to 'BoroughName' and 'All ages' to 'Female_Population'.
    *  Assign the columns denoting populations in 2017-2024 under 'Year'
*   **`rent_df`**:
    *   Rename 'Borough' to 'BoroughName'.
    *   Melt the yearly rent columns (e.g., '2025-2024') into two new columns: 'Year_Range' and 'Rent_Price'.
    *   Extract the end year from 'Year_Range' (e.g., '2024' from '2025-2024') and convert it to an integer, assigning this to a new 'Year' column.

*   **`income_df`**:
    *   Melt the borough-specific income columns (all columns except 'Year') into 'BoroughName' and 'Weekly_Income', retaining the 'Year' column.

*   **`pub_df`**:
    *   Melt the year columns (e.g., '2017') into a 'Year' column and a 'Pub_Count' column, keeping 'BoroughName'.

*   **`sexual_crime_df`**:
    *   Group by 'BoroughName' and 'Year'.
    *   Sum the 'CrimeCount' for each group to get the 'Sexual_Crime_Count' for each borough and year.

*   **`job_df`**:
    *   Rename 'local authority: district / unitary (as of April 2023)' to 'BoroughName'.
    *   Melt the year range columns (e.g., 'Jul 2016-Jun 2017') along with their corresponding 'Conf' columns.
    *   Extract the end year from the melted year ranges (e.g., '2017' from 'Jul 2016-Jun 2017') and convert it to an integer, assigning this to a new 'Year' column.
    *   Drop the 'Conf' columns and rename the value column from the melt operation to 'Unemployment_Rate'.

After preparing each dataframe, perform a series of inner merges on 'BoroughName' and 'Year' to combine them into a single `merged_df`.

Finally, from `merged_df`, select and rename the following columns:
*   'Female_Population' to 'femalepop'
*   'Rent_Price' to 'rent price'
*   'Weekly_Income' to 'weekly income'
*   'Dominant_Ethnicity' to 'ethnicity'
*   'Pub_Count' to 'pub count'
*   'Sexual_Crime_Count' to 'sexual crime'
*   'Unemployment_Rate' to 'unemployment'

Display the head and information (`.info()`) of the final `merged_df`.

In [11]:
# Rename the 'local authority...' column to 'BoroughName'
sex_df = sex_df.rename(columns={'local authority: county / unitary (as of April 2023)': 'BoroughName'})

# Identify columns that represent years for melting
year_cols = [str(year) for year in range(2017, 2025)] # Covers 2017-2024 as per sex_df.head()

# Melt the DataFrame to transform year columns into a single 'Year' column and 'Female_Population'
sex_df_melted = sex_df.melt(
    id_vars=['BoroughName', 'mnemonic'], # Keep BoroughName and mnemonic as identifier variables during melt
    value_vars=year_cols,
    var_name='Year',
    value_name='Female_Population'
)

# Convert the 'Year' column to integer type
sex_df_melted['Year'] = sex_df_melted['Year'].astype(int)

# Update sex_df to be the melted version, dropping the 'mnemonic' column as it's not needed for the final merge
sex_df = sex_df_melted.drop(columns=['mnemonic'])

sex_df.head()

,BoroughName,Year,Female_Population
0,Brent,2017,172035.0
1,Camden,2017,112745.0
2,Greenwich,2017,144020.0
3,Hackney,2017,136757.0
4,Hammersmith and Fulham,2017,97539.0


In [12]:
# Replace the boroughs to BoroughName for consistency
rent_df = rent_df.rename(columns={'Borough': 'BoroughName'})
rent_df.head()

,BoroughName,Mean_Monthly_Rent,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021,2021-2022,2022-2023,2023-2024,2024-2025
0,Brent,1677,1558,1561,1575,1498,NaN,1482,1578,2156,2006
1,Camden,2238,2053,1939,2416,2218,NaN,2076,2383,2202,2615
2,Greenwich,1539,1370,1435,1385,1479,NaN,1478,1557,1740,1866
3,Hackney,1958,1757,1825,1841,1831,NaN,1746,1997,2191,2478
4,Hammersmith and Fulham,2138,1975,1737,2055,2115,NaN,2019,2270,2418,2516


In [13]:
rent_df_melted = rent_df.melt(id_vars=['BoroughName'],
                               value_vars=[col for col in rent_df.columns if '-' in col],
                               var_name='Year_Range',
                               value_name='Rent_Price')
rent_df_melted.head()

,BoroughName,Year_Range,Rent_Price
0,Brent,2016-2017,1558.0
1,Camden,2016-2017,2053.0
2,Greenwich,2016-2017,1370.0
3,Hackney,2016-2017,1757.0
4,Hammersmith and Fulham,2016-2017,1975.0


In [14]:
# Extract end year from year range and convert to an integer to store as the Year column, replacing year range with this
rent_df_melted['Year'] = rent_df_melted['Year_Range'].apply(lambda x: int(x.split('-')[1]))
rent_df_melted = rent_df_melted.drop(columns=['Year_Range'])
rent_df_melted.head()

,BoroughName,Rent_Price,Year
0,Brent,1558.0,2017
1,Camden,2053.0,2017
2,Greenwich,1370.0,2017
3,Hackney,1757.0,2017
4,Hammersmith and Fulham,1975.0,2017


In [15]:
# Melt borough specific income into boroughname and income per year
income_df_melted = income_df.melt(id_vars=['Year'], var_name='BoroughName', value_name='Weekly_Income')
income_df_melted.head()

,Year,BoroughName,Weekly_Income
0,2017,Brent,471.0
1,2018,Brent,479.9
2,2019,Brent,524.0
3,2020,Brent,531.8
4,2021,Brent,555.7


In [16]:
# Transform year columns into 'Year' column and 'Pub Count' column corresponding to BoroughName
pub_df_melted = pub_df.melt(id_vars=['BoroughName'], var_name='Year', value_name='Pub_Count')
pub_df_melted['Year'] = pub_df_melted['Year'].astype(int)
pub_df_melted.head()

,BoroughName,Year,Pub_Count
0,Brent,2017,70
1,Camden,2017,245
2,Greenwich,2017,75
3,Hackney,2017,235
4,Hammersmith and Fulham,2017,95


In [29]:
# Group by BoroughName and Year as these are the constants throughout the dfs, averaging the crime rate for each group
sexual_crime_df_agg = sexual_crime_df.groupby(['BoroughName', 'Year'])['CrimeRate'].mean().reset_index()
sexual_crime_df_agg = sexual_crime_df_agg.rename(columns={'CrimeRate': 'Sexual_Crime_Rate'})
sexual_crime_df_agg.head()

,BoroughName,Year,Sexual_Crime_Rate
0,Brent,2017,0.007556
1,Brent,2018,0.007621
2,Brent,2019,0.007819
3,Brent,2020,0.007122
4,Brent,2021,0.008057


In [31]:
# Do the same for crime_df
crime_df_agg = crime_df.groupby(['BoroughName', 'Year'])['CrimeRate'].mean().reset_index()
crime_df_agg = crime_df_agg.rename(columns={'CrimeRate': 'Crime_Rate'})
crime_df_agg.head()

,BoroughName,Year,Crime_Rate
0,Brent,2017,0.022258
1,Brent,2018,0.022580
2,Brent,2019,0.022401
3,Brent,2020,0.021194
4,Brent,2021,0.020954


In [18]:
# Rename for consistency as done above
job_df = job_df.rename(columns={'local authority: district / unitary (as of April 2023)': 'BoroughName'})
job_df.head()

,BoroughName,mnemonic,Jul 2016-Jun 2017,Conf,Jul 2017-Jun 2018,Conf.1,Jul 2018-Jun 2019,Conf.2,Jul 2019-Jun 2020,Conf.3,Jul 2020-Jun 2021,Conf.4,Jul 2021-Jun 2022,Conf.5,Jul 2022-Jun 2023,Conf.6,Jul 2023-Jun 2024,Conf.7,Jul 2024-Jun 2025,Conf.8
0,Brent,E09000005,6.6,1.5,5.9,1.4,4.1,1.4,5.9,1.9,7.6,2.1,5.4,1.8,7.3,3.1,5.5,2.3,8.2,3.0
1,Camden,E09000007,5.3,1.3,5.0,1.4,4.8,1.6,4.0,1.4,5.3,1.7,3.8,1.4,4.0,1.7,6.3,2.2,5.0,2.4
2,Greenwich,E09000011,6.1,1.4,5.5,1.3,4.3,1.3,4.9,1.4,6.6,1.5,5.9,1.5,4.7,1.5,4.1,1.4,6.4,2.3
3,Hackney,E09000012,5.7,1.4,5.7,1.4,5.0,1.5,5.4,1.6,6.1,1.8,4.3,1.5,4.3,1.7,4.4,1.7,5.3,1.8
4,Hammersmith and Fulham,E09000013,5.8,1.4,5.6,1.5,5.8,1.8,4.5,1.5,5.8,1.7,4.7,1.7,3.9,1.7,5.7,2.1,4.3,1.9


In [19]:
#Identify unemployment rate columns, exclude Conf columns and melt job_df to transform columns to Year Range and Unemployment rate corresponding to Borough Names
unemployment_cols = [col for col in job_df.columns if 'Jul' in col and 'Conf' not in col]
job_df_melted = job_df.melt(id_vars=['BoroughName', 'mnemonic'],
                            value_vars=unemployment_cols,
                            var_name='Year_Range',
                            value_name='Unemployment_Rate')
job_df_melted.head()

,BoroughName,mnemonic,Year_Range,Unemployment_Rate
0,Brent,E09000005,Jul 2016-Jun 2017,6.6
1,Camden,E09000007,Jul 2016-Jun 2017,5.3
2,Greenwich,E09000011,Jul 2016-Jun 2017,6.1
3,Hackney,E09000012,Jul 2016-Jun 2017,5.7
4,Hammersmith and Fulham,E09000013,Jul 2016-Jun 2017,5.8


In [20]:
# Extract end year of year range and assign to new Year column as done above
job_df_melted['Year'] = job_df_melted['Year_Range'].apply(lambda x: int(x.split('-')[1].split(' ')[1]))
job_df_melted.head()

,BoroughName,mnemonic,Year_Range,Unemployment_Rate,Year
0,Brent,E09000005,Jul 2016-Jun 2017,6.6,2017
1,Camden,E09000007,Jul 2016-Jun 2017,5.3,2017
2,Greenwich,E09000011,Jul 2016-Jun 2017,6.1,2017
3,Hackney,E09000012,Jul 2016-Jun 2017,5.7,2017
4,Hammersmith and Fulham,E09000013,Jul 2016-Jun 2017,5.8,2017


In [21]:
# Drop mnemonic and Year Range
job_df_melted = job_df_melted.drop(columns=['Year_Range', 'mnemonic'])
job_df_melted.head()

,BoroughName,Unemployment_Rate,Year
0,Brent,6.6,2017
1,Camden,5.3,2017
2,Greenwich,6.1,2017
3,Hackney,5.7,2017
4,Hammersmith and Fulham,5.8,2017


In [24]:
import pandas as pd
from itertools import product

# Clean 'BoroughName' in each dataframe by stripping whitespace
sex_df['BoroughName'] = sex_df['BoroughName'].str.strip()
rent_df_melted['BoroughName'] = rent_df_melted['BoroughName'].str.strip()
income_df_melted['BoroughName'] = income_df_melted['BoroughName'].str.strip()
pub_df_melted['BoroughName'] = pub_df_melted['BoroughName'].str.strip()
sexual_crime_df_agg['BoroughName'] = sexual_crime_df_agg['BoroughName'].str.strip()
job_df_melted['BoroughName'] = job_df_melted['BoroughName'].str.strip()

# Consolidate all unique BoroughNames from all dataframes
all_boroughs = pd.concat([
    sex_df['BoroughName'],
    rent_df_melted['BoroughName'],
    income_df_melted['BoroughName'],
    pub_df_melted['BoroughName'],
    sexual_crime_df_agg['BoroughName'],
    job_df_melted['BoroughName']
]).unique()

# Filter out non-borough entries from all_boroughs
non_borough_entries = ['Column Total', 'Population estimates - local authority based by five year age band'] # Add any other identified non-borough entries
all_boroughs = [b for b in all_boroughs if b not in non_borough_entries]

# Define the years to include
years_to_include = pd.Series(range(2017, 2025)) # 2017 to 2024 inclusive

# Create a master dataframe with all borough-year combinations
master_combinations = pd.DataFrame(list(product(all_boroughs, years_to_include)), columns=['BoroughName', 'Year'])

# Start merged_df with the master combinations
merged_df = master_combinations

# Perform left merges with each prepared dataframe
merged_df = pd.merge(merged_df, sex_df, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, rent_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, income_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, pub_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, sexual_crime_df_agg, on=['BoroughName', 'Year'], how='left')

merged_df = pd.merge(merged_df, job_df_melted, on=['BoroughName', 'Year'], how='left')

# Rename columns as specified in the task
merged_df = merged_df.rename(columns={
    'Female_Population': 'femalepop',
    'Rent_Price': 'rent price',
    'Weekly_Income': 'weekly income',
    'Pub_Count': 'pub count',
    'Sexual_Crime_Rate': 'sexual crime',
    'Unemployment_Rate': 'unemployment'
})

# Display head and info of the final merged_df
display(merged_df.head())
display(merged_df.info())

,BoroughName,Year,femalepop,rent price,weekly income,pub count,sexual crime,unemployment
0,Brent,2017,172035.0,1558.0,471.0,70.0,0.181346,6.6
1,Brent,2018,174420.0,1561.0,479.9,65.0,0.182893,5.9
2,Brent,2019,175791.0,1575.0,524.0,65.0,0.187667,4.1
3,Brent,2020,174877.0,1498.0,531.8,55.0,0.170936,5.9
4,Brent,2021,172954.0,NaN,555.7,60.0,0.193369,7.6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   BoroughName    96 non-null     object 
 1   Year           96 non-null     int64  
 2   femalepop      88 non-null     float64
 3   rent price     77 non-null     float64
 4   weekly income  88 non-null     float64
 5   pub count      77 non-null     float64
 6   sexual crime   88 non-null     float64
 7   unemployment   88 non-null     float64
dtypes: float64(6), int64(1), object(1)
memory usage: 6.1+ KB


None

In [25]:
merged_df.tail()

,BoroughName,Year,femalepop,rent price,weekly income,pub count,sexual crime,unemployment
91,Westminister,2020,NaN,NaN,NaN,425.0,NaN,NaN
92,Westminister,2021,NaN,NaN,NaN,430.0,NaN,NaN
93,Westminister,2022,NaN,NaN,NaN,425.0,NaN,NaN
94,Westminister,2023,NaN,NaN,NaN,435.0,NaN,NaN
95,Westminister,2024,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#Convert to csv and download
merged_df.to_csv('merged_data with 24 pub count.csv')

In [ ]:
files.download('merged_data with 24 pub count.csv')

# Comparison df to general crime

To ensure that we are not conflating an strong correlation with general crime rather than sexual crime, we prepare another merged df for crime per borough.

In [32]:
import pandas as pd
from itertools import product

# Clean 'BoroughName' in each dataframe by stripping whitespace
sex_df['BoroughName'] = sex_df['BoroughName'].str.strip()
rent_df_melted['BoroughName'] = rent_df_melted['BoroughName'].str.strip()
income_df_melted['BoroughName'] = income_df_melted['BoroughName'].str.strip()
pub_df_melted['BoroughName'] = pub_df_melted['BoroughName'].str.strip()
crime_df_agg['BoroughName'] = crime_df_agg['BoroughName'].str.strip()
job_df_melted['BoroughName'] = job_df_melted['BoroughName'].str.strip()

# Consolidate all unique BoroughNames from all dataframes
all_boroughs = pd.concat([
    sex_df['BoroughName'],
    rent_df_melted['BoroughName'],
    income_df_melted['BoroughName'],
    pub_df_melted['BoroughName'],
    crime_df_agg['BoroughName'],
    job_df_melted['BoroughName']
]).unique()

# Filter out non-borough entries from all_boroughs
non_borough_entries = ['Column Total', 'Population estimates - local authority based by five year age band'] # Add any other identified non-borough entries
all_boroughs = [b for b in all_boroughs if b not in non_borough_entries]

# Define the years to include
years_to_include = pd.Series(range(2017, 2025)) # 2017 to 2024 inclusive

# Create a master dataframe with all borough-year combinations
master_combinations = pd.DataFrame(list(product(all_boroughs, years_to_include)), columns=['BoroughName', 'Year'])

# Start merged_df with the master combinations
merged_df = master_combinations

# Perform left merges with each prepared dataframe
merged_df = pd.merge(merged_df, sex_df, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, rent_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, income_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, pub_df_melted, on=['BoroughName', 'Year'], how='left')
merged_df = pd.merge(merged_df, crime_df_agg, on=['BoroughName', 'Year'], how='left')

merged_df = pd.merge(merged_df, job_df_melted, on=['BoroughName', 'Year'], how='left')

# Rename columns as specified in the task
merged_df = merged_df.rename(columns={
    'Female_Population': 'femalepop',
    'Rent_Price': 'rent price',
    'Weekly_Income': 'weekly income',
    'Pub_Count': 'pub count',
    'Crime_Rate': 'crime rate',
    'Unemployment_Rate': 'unemployment'
})

# Display head and info of the final merged_df
display(merged_df.head())
display(merged_df.info())

,BoroughName,Year,femalepop,rent price,weekly income,pub count,crime rate,unemployment
0,Brent,2017,172035.0,1558.0,471.0,70.0,0.022258,6.6
1,Brent,2018,174420.0,1561.0,479.9,65.0,0.022580,5.9
2,Brent,2019,175791.0,1575.0,524.0,65.0,0.022401,4.1
3,Brent,2020,174877.0,1498.0,531.8,55.0,0.021194,5.9
4,Brent,2021,172954.0,NaN,555.7,60.0,0.020954,7.6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   BoroughName    96 non-null     object 
 1   Year           96 non-null     int64  
 2   femalepop      88 non-null     float64
 3   rent price     77 non-null     float64
 4   weekly income  88 non-null     float64
 5   pub count      77 non-null     float64
 6   crime rate     88 non-null     float64
 7   unemployment   88 non-null     float64
dtypes: float64(6), int64(1), object(1)
memory usage: 6.1+ KB


None

In [33]:
#Convert to csv and download
merged_df.to_csv('merged_data_crime.csv')

In [34]:
files.download('merged_data_crime.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>